In [ ]:
import numpy as np
import pandas as pd
df1 = pd.read_csv('calendar.csv')
df2 = pd.read_csv('sample_submission.csv')
df3 = pd.read_csv('sell_prices.csv')
df4 = pd.read_csv('sales_train_validation.csv')
df5 = pd.read_csv('sales_test_validation.csv')

In [ ]:
#Investigate price dataset
import matplotlib.pyplot as plt

itemid = 'FOODS_3_007'
filtered_df3 = df3[df3['item_id'] == itemid]

# Scatter plot of price vs time
plt.figure(figsize=(10, 6))
plt.scatter(filtered_df3['wm_yr_wk'], filtered_df3['sell_price'], marker='o')
title = 'Sell Price Over Time for '+ itemid
plt.title(title)
plt.xlabel('Time (wm_yr_wk)')
plt.ylabel('Sell Price')
plt.grid(True)
plt.show()

Manual inspection concludes there is no trend in price of products. Outliers are clear, may reflect promotions. Although event information is given, we chose to disregard all event information by removing outliers from sales quantity data as well.

In [ ]:
first_row = df4.loc[3].drop(labels=['id'])
time_index = range(1, len(first_row) + 1)

#Scatter plot for food 1 sale
plt.figure(figsize=(10, 6))
plt.scatter(time_index, first_row, marker='o')
plt.title('Quantity Over Time for First Row of df4')
plt.xlabel('Time')
plt.ylabel('Quantity')
plt.grid(True)
plt.show()


In [8]:
#join train and test validation set, we'll use test evaluation data as validation with our first approach
df4 = pd.concat([df4,df5.iloc[:,1:]],axis=1)

In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor

def predict_single_row_GBR(df_sale, df_cal, rowid):
    food_id = df_sale.iloc[rowid,0]
    row = df_sale.iloc[rowid,1:]
    
    # Drop all columns from the beginning where the value is 0 until the first non-zero value
    first_non_zero_index = row.ne(0).idxmax()
    filtered_sales = row[first_non_zero_index:]

    # Count the length of the days where this product existed
    num_days_existed = len(filtered_sales)

    # Select the matching rows from df1 by counting backwards from the last row - 28
    start_index = len(df_cal) - 28 - num_days_existed
    selected_calendar = df_cal.iloc[start_index:len(df_cal) - 28]

    # Create a new dataframe with the sales quantity data as a new column
    result_df = selected_calendar.copy()
    result_df['sales_quantity'] = filtered_sales.values
    result_df = result_df.drop(columns=['date','wm_yr_wk','wday','year','event_name_1','event_name_2','event_type_2'])
    
    # Convert categorical features to numerical using one-hot encoding
    categorical_features = ['weekday', 'month', 'event_type_1']
    categories = [ ['Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], # weekday categories 
                  list(range(1, 13)), # month categories (1 to 12) 
                  result_df['event_type_1'].unique().tolist() # unique event_type_1 categories
                 ]
    encoder = OneHotEncoder(sparse=False, drop='first', categories=categories)
    encoded_features = encoder.fit_transform(result_df[categorical_features])

    # Create a new dataframe with encoded features
    encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))
    result_df = result_df.drop(columns=categorical_features).reset_index(drop=True)
    result_df = pd.concat([result_df, encoded_df], axis=1)
    
    # Split the data into training and testing sets
    X_train = result_df.drop(columns=['sales_quantity'])
    y_train = result_df['sales_quantity']

    # Build and train the model
    model = GradientBoostingRegressor()
    model.fit(X_train, y_train)
    
    # Process test data
    X_test = df_cal.tail(28).drop(columns=['date','wm_yr_wk','wday','year','event_name_1','event_name_2','event_type_2'])
    encoded_features = encoder.transform(X_test[categorical_features]) # does not fit again
    encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))
    X_test = X_test.drop(columns=categorical_features).reset_index(drop=True)
    X_test = pd.concat([X_test, encoded_df], axis=1)
    
    # Predict
    y_pred = model.predict(X_test)
    
    # Convert to integer
    y_pred = np.rint(y_pred).astype(int)
    y_pred = np.maximum(y_pred, 0)

    return food_id, y_pred
    

In [ ]:
result_pred = pd.DataFrame(columns=df2.columns)

for row_id in range(len(df4)):
    food_id, pred = predict_single_row_GBR(df_sale=df4, df_cal=df1, rowid=row_id)
    series = pd.concat([pd.Series([food_id]), pd.Series(pred)], ignore_index=True)
    result_pred = pd.concat([result_pred, pd.DataFrame([series])], axis=0, ignore_index=True)
    print(str(row_id) + ' done')


In [11]:
result_pred = result_pred.iloc[:,29:]
result_pred.columns = df2.columns

In [12]:
from sklearn.metrics import mean_squared_error

df_eval = pd.read_csv('sales_test_evaluation_afcs_2024.csv')
pred_numerical = result_pred.iloc[:, 1:]
eval_numerical = df_eval.iloc[:, 1:]


# RMSE for every day
rmse_values = {}
for i in range(pred_numerical.shape[1]):
    rmse = np.sqrt(mean_squared_error(pred_numerical.iloc[:, i], eval_numerical.iloc[:, i]))
    rmse_values[i] = rmse

for column, rmse in rmse_values.items():
    print(f'RMSE for {column}: {rmse}')

total_rmse = sum(rmse_values.values())
print(f'Total RMSE: {total_rmse}')

average_rmse = total_rmse / len(rmse_values)
print(f'Average RMSE per day: {average_rmse}')


RMSE for 0: 3.4866522081932905
RMSE for 1: 5.234013442791254
RMSE for 2: 2.63424498833465
RMSE for 3: 2.5881776265334757
RMSE for 4: 3.394298700689411
RMSE for 5: 4.51916399972153
RMSE for 6: 3.3237610633615566
RMSE for 7: 3.41535379650272
RMSE for 8: 2.5555790349999405
RMSE for 9: 3.534072994712511
RMSE for 10: 3.1185565405679787
RMSE for 11: 3.3889248537908827
RMSE for 12: 3.157663439334817
RMSE for 13: 4.211740807769959
RMSE for 14: 3.7659341502874604
RMSE for 15: 3.859784240911822
RMSE for 16: 3.5022991927418126
RMSE for 17: 3.5258117822963593
RMSE for 18: 3.5316654706179067
RMSE for 19: 4.161969811231098
RMSE for 20: 4.872368592241706
RMSE for 21: 4.136492606231695
RMSE for 22: 4.508396378989436
RMSE for 23: 4.074935984856748
RMSE for 24: 3.669022403160918
RMSE for 25: 4.092787717451457
RMSE for 26: 3.902985117971334
RMSE for 27: 4.722557743462775
Total RMSE: 104.88921468975653
Average RMSE per day: 3.7460433817770187


In [13]:
# RMSE for every item
rmse_values = {}
for i in range(pred_numerical.shape[0]):
    rmse = np.sqrt(mean_squared_error(pred_numerical.iloc[i, :], eval_numerical.iloc[i, :]))
    rmse_values[i] = rmse

for row_index, rmse in rmse_values.items():
    print(f'RMSE for food item {row_index}: {rmse}')

total_rmse = sum(rmse_values.values())
print(f'Total RMSE: {total_rmse}')

average_rmse = total_rmse / len(rmse_values)
print(f'Average RMSE per item: {average_rmse}')


RMSE for food item 0: 1.0690449676496976
RMSE for food item 1: 2.0615528128088303
RMSE for food item 2: 0.944911182523068
RMSE for food item 3: 0.7071067811865476
RMSE for food item 4: 1.5923926292577109
RMSE for food item 5: 0.9063269671749657
RMSE for food item 6: 6.777062153217383
RMSE for food item 7: 7.794228634059948
RMSE for food item 8: 0.9819805060619657
RMSE for food item 9: 0.7559289460184544
RMSE for food item 10: 1.118033988749895
RMSE for food item 11: 1.0690449676496976
RMSE for food item 12: 1.4015297764534702
RMSE for food item 13: 1.3627702877384937
RMSE for food item 14: 2.434865792722759
RMSE for food item 15: 1.2535663410560174
RMSE for food item 16: 0.3779644730092272
RMSE for food item 17: 0.8864052604279183
RMSE for food item 18: 1.6903085094570331
RMSE for food item 19: 3.1679195516480982
RMSE for food item 20: 2.0788046015507495
RMSE for food item 21: 0.8864052604279183
RMSE for food item 22: 1.2100767390069582
RMSE for food item 23: 5.800246300189083
RMSE for